In [1]:
using Pkg
Pkg.add(["Genie", "HTTP", "JSON", "CSV", "DataFrames"])

   Resolving package versions...
  No Changes to `C:\Users\JANA\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\JANA\.julia\environments\v1.11\Manifest.toml`


In [2]:
using Genie, Genie.Router, Genie.Renderer.Json, HTTP, JSON, CSV, DataFrames

In [3]:
Genie.config.server_host = "0.0.0.0"
Genie.config.server_port = 8000
Genie.config.run_as_server = true

true

In [4]:
function load_earthquake_data()
    file_path = "phivolcs_earthquake_data.csv" 
    data = CSV.File(file_path) |> DataFrame
    return data[:, [:Date_Time_PH, :Latitude, :Longitude, :Depth_In_Km, :Magnitude, :Location]]  # Select relevant columns
end

load_earthquake_data (generic function with 1 method)

In [5]:
df = CSV.File("phivolcs_earthquake_data.csv") |> DataFrame
println(first(df, 5))

5×6 DataFrame
 Row │ Date_Time_PH         Latitude  Longitude  Depth_In_Km  Magnitude  Location                          
     │ String31             String7   String7    String7      Float64    String                            
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 2016-01-01 00:40:00  17.34     120.30     023                3.0  015 km N 87° W of San Esteban (I…
   2 │ 2016-01-01 05:06:00  14.65     123.12     017                3.3  054 km N 42° E of Paracale (Cama…
   3 │ 2016-01-01 13:24:00  09.76     125.46     012                2.4  005 km S 42° W of Surigao City
   4 │ 2016-01-01 15:01:00  17.30     120.27     026                2.9  018 km S 81° W of San Esteban (I…
   5 │ 2016-01-01 20:27:00  08.89     126.28     024                3.0  004 km S 44° W of Cagwait (Surig…


In [6]:
route("/api/earthquakes", method = GET) do
    data = load_earthquake_data()
    JSON.json(collect(eachrow(data)))
end

[GET] /api/earthquakes => #11 | :get_api_earthquakes

In [7]:
route("/") do
    html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Philippines Earthquake Map</title>
        <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
        <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
        <style>
            body { font-family: Arial, sans-serif; text-align: center; }
            #map { height: 500px; width: 80%; margin: 20px auto; border: 1px solid #ccc; }
            #controls { margin-bottom: 10px; padding: 10px; background: #f8f9fa; border-radius: 5px; }
            select, button { margin: 5px; padding: 5px; }
            button { background: #007bff; color: white; border: none; cursor: pointer; }
        </style>
    </head>
    <body>
        <h1>Philippines Earthquake Map</h1>
        <div id="controls">
            <label for="yearFilter">Year:</label>
            <select id="yearFilter"></select>

            <label for="locationFilter">Location:</label>
            <select id="locationFilter">
                <option value="">All Locations</option>
            </select>

            <button onclick="filterEarthquakes()">Filter</button>
        </div>
        <div id="map"></div>
        
        <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
        <script>
            var map = L.map('map').setView([12.8797, 121.7740], 6);
            L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png').addTo(map);
            var markers = [];

            // Populate Year Dropdown
            let yearDropdown = document.getElementById("yearFilter");
            for (let year = 2016; year <= 2023; year++) {
                let option = document.createElement("option");
                option.value = year;
                option.text = year;
                yearDropdown.appendChild(option);
            }

            // Fetch and Populate Location Dropdown
            function loadLocations() {
                fetch('/api/locations')
                    .then(response => response.json())
                    .then(locations => {
                        let locationDropdown = document.getElementById('locationFilter');
                        locationDropdown.innerHTML = '<option value="">All Locations</option>';
                        locations.forEach(location => {
                            if (location.trim() !== "") { // Avoid empty values
                                let option = document.createElement('option');
                                option.value = location;
                                option.textContent = location;
                                locationDropdown.appendChild(option);
                            }
                        });
                    })
                    .catch(error => console.error("Error fetching locations:", error));
            }

            function loadEarthquakes() {
                fetch('/api/earthquakes')
                    .then(response => response.json())
                    .then(data => {
                        console.log("Fetched Data:", data);

                        markers.forEach(marker => map.removeLayer(marker));
                        markers = [];

                        let selectedYear = parseInt(document.getElementById('yearFilter').value) || 0;
                        let selectedLocation = document.getElementById('locationFilter').value;

                        data.forEach(eq => {
                            let eqYear = new Date(eq.Date_Time_PH).getFullYear();
                            let lat = Number(eq.Latitude);
                            let lon = Number(eq.Longitude);
                            let mag = Number(eq.Magnitude);
                            let depth = Number(eq.Depth_In_Km);
                            let loc = String(eq.Location);

                            if (!isNaN(lat) && !isNaN(lon) &&
                                eqYear === selectedYear &&
                                (selectedLocation === "" || loc === selectedLocation)) {
                                
                                let marker = L.marker([lat, lon], {
                                    icon: L.icon({
                                        iconUrl: 'https://upload.wikimedia.org/wikipedia/commons/e/ec/RedDot.svg',
                                        iconSize: [8, 8] 
                                    })
                                })
                                .bindPopup(\`
                                    <b>Magnitude:</b> \${mag}<br>
                                    <b>Depth:</b> \${depth} km<br>
                                    <b>Location:</b> \${loc}
                                \`).addTo(map);
                                
                                markers.push(marker);
                            }
                        });
                    })
                    .catch(error => console.error("Error fetching earthquake data:", error));
            }

            function filterEarthquakes() {
                loadEarthquakes();
            }

            loadEarthquakes();
        </script>
    </body>
    </html>
    """
    html
end

[GET] / => #13 | :get

In [ ]:
Genie.up()

┌ Info: 
│ Web Server starting at http://0.0.0.0:8000 - press Ctrl/Cmd+C to stop the server. 
└ @ Genie.Server C:\Users\JANA\.julia\packages\Genie\dFkvA\src\Server.jl:182
┌ Info: Listening on: 0.0.0.0:8000, thread id: 1
└ @ HTTP.Servers C:\Users\JANA\.julia\packages\HTTP\4AUPl\src\Servers.jl:382
┌ Info: GET / 200
└ @ Genie.Router C:\Users\JANA\.julia\packages\Genie\dFkvA\src\Router.jl:201
┌ Info: GET /api/earthquakes 200
└ @ Genie.Router C:\Users\JANA\.julia\packages\Genie\dFkvA\src\Router.jl:201
